In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot


pyplot.style.use('seaborn')

In [ ]:
time_period = 0.00001
mass = 0.110
moment_of_inertia = (mass * 0.05 ** 2) / 2
wheels_separation = 0.065

In [ ]:
from numpy import cos
from numpy import pi
from numpy import sin
from numpy import sign
from numpy import sqrt
from pandas import DataFrame


def dynamic_profile(linear_speed, radius, transition, arc, time_period):
    angular_velocity = [0]
    distance = [linear_speed * time_period]
    max_angular_velocity = linear_speed / radius

    while True:
        if distance[-1] >= 2 * transition + arc:
            break
        new_angular_velocity = max_angular_velocity
        if distance[-1] < transition:
            factor = distance[-1] / transition
            new_angular_velocity *= sin(factor * pi / 2)
        elif distance[-1] >= transition + arc:
            factor = (distance[-1] - arc) / transition
            new_angular_velocity *= sin(factor * pi / 2)
        angular_velocity.append(new_angular_velocity)
        distance.append(distance[-1] + linear_speed * time_period)
    columns = {
        'angular_velocity': angular_velocity,
        'linear_velocity': linear_speed,
        'period': time_period,
    }
    return DataFrame(columns)


def static_profile(total_angle, force, wheels_separation, moment_of_inertia, max_angular_velocity, time_period):
    turn_sign = sign(total_angle)
    total_angle = abs(total_angle)
    angular_acceleration = force * wheels_separation / moment_of_inertia
    max_angular_velocity = min(max_angular_velocity,
                               sqrt(total_angle / 2 * angular_acceleration))
    duration = max_angular_velocity / angular_acceleration * pi
    transition_angle = duration * max_angular_velocity / pi
    arc = (total_angle - 2 * transition_angle) / max_angular_velocity
    transition = duration / 2
    max_angular_velocity = max_angular_velocity * turn_sign

    angular_velocity = [0]
    angle = [0]
    time = [0]

    while True:
        if time[-1] >= 2 * transition + arc:
            break
        new_angular_velocity = max_angular_velocity
        if time[-1] < transition:
            factor = time[-1] / transition
            new_angular_velocity *= sin(factor * pi / 2)
        elif time[-1] >= transition + arc:
            factor = (time[-1] - arc) / transition
            new_angular_velocity *= sin(factor * pi / 2)
        angular_velocity.append(new_angular_velocity)
        time.append(time[-1] + time_period)
        angle.append(angle[-1] + angular_velocity[-1] * time_period)

    columns = {
        'angular_velocity': angular_velocity,
        'linear_velocity': 0,
        'period': time_period,
        'angle': angle,
        
    }
    return DataFrame(columns)


def complete_profile(profile, mass, moment_of_inertia, wheels_separation):
    profile['angle'] = (profile['angular_velocity'] * profile['period']).cumsum()
    profile['x'] = (profile['linear_velocity'] * cos(profile['angle']) * profile['period']).cumsum()
    profile['y'] = (profile['linear_velocity'] * sin(profile['angle']) * profile['period']).cumsum()
    profile['centrifugal_force'] = mass * profile['linear_velocity'] * profile['angular_velocity'] / 2
    angular_acceleration = profile['angular_velocity'].diff() / profile['period']
    profile['angular_acceleration_force'] = (moment_of_inertia * angular_acceleration / wheels_separation).abs()
    profile['total_force'] = sqrt(profile['centrifugal_force'] ** 2 + profile['angular_acceleration_force'] ** 2)
    profile['time'] = profile['period'].cumsum()
    profile = profile.set_index('time')
    return profile


def plot_profile(profile):
    x, y = profile['x'], profile['y']
    print('Final position: (%.5f, %.5f)' % (x.iloc[-1], y.iloc[-1]))
    
    profile[['centrifugal_force', 'angular_acceleration_force', 'total_force']].plot(style='.-')
    profile[['angular_velocity']].plot(style='.-')

    pyplot.figure(figsize=(6, 6))
    pyplot.plot(x, y, '.-')
    pyplot.show()

## Static turns

In [ ]:
max_angular_velocity = 10.
profile = static_profile(
    pi / 2,
    0.25,
    wheels_separation,
    moment_of_inertia,
    max_angular_velocity,
    time_period)

profile = complete_profile(profile, mass, moment_of_inertia, wheels_separation)
plot_profile(profile)
profile[['angle']].plot(style='.-')

## Search turn

In [ ]:
params = {
    'linear_speed': .4627487,
    'transition': 0.05812,
    'radius': 0.04711,
    'arc': 0.,
    'time_period': time_period,
}
profile = dynamic_profile(**params)
profile = complete_profile(profile, mass, moment_of_inertia, wheels_separation)
plot_profile(profile)

## 90º turn

In [ ]:
params = {
    'linear_speed': .8,
    'angular_acceleration': 32 * pi,
    'max_angular_velocity': 2.3 * pi,
    'turn_angle': pi / 2,
}


plot_turn(**params)

## 180º turn

In [ ]:
params = {
    'linear_speed': .7,
    'angular_acceleration': 32 * pi,
    'max_angular_velocity': 2.5 * pi,
    'turn_angle': pi,
}


plot_turn(**params)

# Turn to 45º

In [ ]:
params = {
    'linear_speed': .71,
    'angular_acceleration': 32 * pi,
    'max_angular_velocity': 2. * pi,
    'turn_angle': pi / 4.,
}


plot_turn(**params)

In [ ]:
before = 0.04978 - 0.11824
before

In [ ]:
after = (2 * (0.09 - 0.04978) ** 2) ** 0.5
after

## Turn to 135º

In [ ]:
params = {
    'linear_speed': .6,
    'angular_acceleration': 32 * pi,
    'max_angular_velocity': 2.5 * pi,
    'turn_angle': 3 * pi / 4.,
}


plot_turn(**params)

In [ ]:
before = (0.18 - 0.14936) - 0.06075
before

In [ ]:
after = (2 * (0.18 - 0.14936) ** 2) ** 0.5
after

## Turn diagonal 90º

In [ ]:
params = {
    'linear_speed': .6,
    'angular_acceleration': 32 * pi,
    'max_angular_velocity': 2.5 * pi,
    'turn_angle': pi / 2.,
}


plot_turn(**params)

Side:

In [ ]:
(2 * 0.09 ** 2) ** 0.5